# LLaMA 4 — Benchmark QCAL Coherence Evaluation

Este notebook benchmarking evalúa las respuestas de LLaMA 4 usando las métricas QCAL:
- **Ψ**: Coherencia cuántica (I × A²_eff)
- **SNR**: Signal-to-Noise Ratio (unique/total words)
- **KLD⁻¹**: Inverso de divergencia Kullback-Leibler
- **∴**: Tasa de símbolos lógicos (strich_rate)

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

## 1. Cargar Resultados

In [ ]:
# Cargar resultados de evaluación
results_path = Path('../results_llama4.json')
if results_path.exists():
    with open(results_path) as f:
        results = json.load(f)
    df = pd.DataFrame(results)
    print(f"✓ Cargados {len(df)} resultados")
    display(df.head())
else:
    print("⚠️ No se encontró results_llama4.json. Ejecuta scripts/qcal_llm_eval.py primero.")

## 2. Estadísticas Descriptivas

In [ ]:
if 'df' in locals():
    print("\n=== Estadísticas de Métricas QCAL ===")
    print(df[['psi', 'snr', 'kld', 'strich_rate']].describe())
    
    # Calcular KLD⁻¹ para análisis
    df['kld_inv'] = 1 / df['kld']
    print(f"\nΨ promedio: {df['psi'].mean():.3f}")
    print(f"SNR promedio: {df['snr'].mean():.3f}")
    print(f"KLD⁻¹ promedio: {df['kld_inv'].mean():.3f}")
    print(f"∴ promedio: {df['strich_rate'].mean():.4f}")

## 3. Visualización: Distribución de Ψ

In [ ]:
if 'df' in locals():
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.hist(df['psi'], bins=20, color='#4CAF50', alpha=0.7, edgecolor='black')
    ax.axvline(df['psi'].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df["psi"].mean():.3f}')
    ax.set_xlabel('Ψ (Coherencia Cuántica)', fontsize=12)
    ax.set_ylabel('Frecuencia', fontsize=12)
    ax.set_title('Distribución de Coherencia Ψ — LLaMA 4', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('histograma_psi_llama4.png', dpi=150)
    plt.show()
    print("✓ Guardado: histograma_psi_llama4.png")

## 4. Visualización: KLD⁻¹ vs SNR

In [ ]:
if 'df' in locals():
    fig, ax = plt.subplots(figsize=(10, 6))
    
    scatter = ax.scatter(df['kld_inv'], df['snr'], c=df['psi'], cmap='viridis', 
                         s=100, alpha=0.7, edgecolor='black')
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Ψ', fontsize=12)
    
    ax.set_xlabel('KLD⁻¹ (Diversidad Lingüística)', fontsize=12)
    ax.set_ylabel('SNR (Relación Señal-Ruido)', fontsize=12)
    ax.set_title('Relación KLD⁻¹ vs SNR — LLaMA 4', fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3)
    
    # Anotar puntos
    for idx, row in df.iterrows():
        ax.annotate(row['prompt'], (row['kld_inv'], row['snr']), 
                   fontsize=8, alpha=0.7, ha='right')
    
    plt.tight_layout()
    plt.savefig('scatter_kld_snr_llama4.png', dpi=150)
    plt.show()
    print("✓ Guardado: scatter_kld_snr_llama4.png")

## 5. Visualización: ∴-rate por Prompt

In [ ]:
if 'df' in locals():
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.barh(df['prompt'], df['strich_rate'], color='#2196F3', alpha=0.7, edgecolor='black')
    ax.set_xlabel('∴ Rate', fontsize=12)
    ax.set_ylabel('Prompt', fontsize=12)
    ax.set_title('Tasa de Símbolos Lógicos por Prompt — LLaMA 4', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('barras_strich_rate_llama4.png', dpi=150)
    plt.show()
    print("✓ Guardado: barras_strich_rate_llama4.png")

## 6. Exportar Resultados

In [ ]:
if 'df' in locals():
    # Exportar CSV
    csv_path = 'resultados_llama4_qcal.csv'
    df.to_csv(csv_path, index=False)
    print(f"✓ Exportado: {csv_path}")
    
    # Generar resumen PDF (placeholder)
    print("\n📊 Para generar PDF, instalar: pip install fpdf")
    print("   Luego usar FPDF o reportlab para crear informe con gráficos.")

## 7. Resumen Ejecutivo

In [ ]:
if 'df' in locals():
    print("\n" + "="*70)
    print("RESUMEN EJECUTIVO — LLaMA 4 QCAL Benchmark")
    print("="*70)
    print(f"Total de prompts evaluados: {len(df)}")
    print(f"\nMétricas Promedio:")
    print(f"  • Ψ (Coherencia):          {df['psi'].mean():.3f}")
    print(f"  • SNR:                      {df['snr'].mean():.3f}")
    print(f"  • KLD⁻¹:                    {df['kld_inv'].mean():.3f}")
    print(f"  • ∴ Rate:                   {df['strich_rate'].mean():.4f}")
    print(f"\nCriterio de Éxito (Ψ ≥ 5.0): {'✓ APROBADO' if df['psi'].mean() >= 5.0 else '✗ NO APROBADO'}")
    print("="*70)

## 8. Comparación con Otros Modelos (Opcional)

Para comparar con GPT-4 o Claude, cargar sus resultados en formato similar y agregar columnas comparativas.

## 9. Subida a Zenodo (Opcional)

```bash
# Instalar zenodo-client
pip install zenodo-client

# Configurar token
export ZENODO_TOKEN=your_token_here

# Subir archivos
zenodo upload resultados_llama4_qcal.csv histograma_psi_llama4.png scatter_kld_snr_llama4.png
```